In [1]:
import pandas as pd
import numpy as np
import os

from tqdm import tqdm

import nibabel as nib

import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image


import sys
import torch
from torch import Tensor
import torchvision.transforms as transforms
from torchvision.transforms import Resize, ToTensor
# from models import *

%load_ext autoreload
%autoreload 2

sys.path.insert(0,'/home/roshansk/Covid/CXRData/')
from SegLearner import *

import segmentation_models_pytorch as smp

sys.path.insert(0,'/home/roshansk/Covid/RibFrac/Models/')
from models import *

from monai.transforms import AddChannel, AdjustContrast, Resize, LoadNifti, LoadNiftiD, Compose, ToTensor

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.


In [2]:
sys.path.insert(0,'/home/roshansk/Covid/3dSeg/Liver/3D/3D_UNet/pytorch_3D-Unet-master/unet3d/')
from model import *

### Data

In [3]:
### Decathlon Dataset
trainFolder = '/data2/roshansk/Liver/Task03_Liver/'

imgFiles = os.listdir(os.path.join(trainFolder, 'imagesTr'))

maskFiles = os.listdir(os.path.join(trainFolder, 'labelsTr'))

imgFiles = [x for x in imgFiles if x[0]!='.'] 
maskFiles = [x for x in maskFiles if x[0]!='.'] 

df = pd.DataFrame({'imgPath':imgFiles, 'maskPath':maskFiles})

df.imgPath = df.imgPath.apply(lambda x : os.path.join(trainFolder, 'imagesTr', x))
df.maskPath = df.maskPath.apply(lambda x : os.path.join(trainFolder, 'labelsTr',x))

## Train Test Split

trainDf = df.iloc[:104]
testDf = df.iloc[104:]


df.head()

,imgPath,maskPath
0,/data2/roshansk/Liver/Task03_Liver/imagesTr/li...,/data2/roshansk/Liver/Task03_Liver/labelsTr/li...
1,/data2/roshansk/Liver/Task03_Liver/imagesTr/li...,/data2/roshansk/Liver/Task03_Liver/labelsTr/li...
2,/data2/roshansk/Liver/Task03_Liver/imagesTr/li...,/data2/roshansk/Liver/Task03_Liver/labelsTr/li...
3,/data2/roshansk/Liver/Task03_Liver/imagesTr/li...,/data2/roshansk/Liver/Task03_Liver/labelsTr/li...
4,/data2/roshansk/Liver/Task03_Liver/imagesTr/li...,/data2/roshansk/Liver/Task03_Liver/labelsTr/li...


In [21]:
### IRCAD Dataset
trainFolder = '/home/roshansk/Covid/3dSeg/Liver/Data/OrigData/'

dataFiles = os.listdir(trainFolder)
imgFiles = [x for x in dataFiles if 'orig' in x]
maskFiles = [x for x in dataFiles if 'liver' in x]


df = pd.DataFrame({'imgPath':imgFiles, 'maskPath':maskFiles})
df.imgPath = df.imgPath.apply(lambda x : os.path.join(trainFolder, x))
df.maskPath = df.maskPath.apply(lambda x : os.path.join(trainFolder, x))


## Train Test Split

trainDf = df.iloc[:16]
testDf = df.iloc[16:]


df.head()

,imgPath,maskPath
0,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...
1,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...
2,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...
3,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...
4,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...,/home/roshansk/Covid/3dSeg/Liver/Data/OrigData...


In [4]:
df.maskPath.iloc[0]

'/data2/roshansk/Liver/Task03_Liver/labelsTr/liver_25.nii.gz'

In [5]:
img = LoadNifti(image_only=True)(df.maskPath.iloc[0])

In [7]:
pd.Series(img.reshape(-1)).value_counts()

0.0    154523331
1.0      3024759
2.0          454
dtype: int64

In [10]:
mask = img==2

In [12]:
mask.astype('float')

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

### Util Functions

In [13]:
class SegDataset3D(torch.utils.data.Dataset):
    
  def __init__(self, df, imgTransforms = None, maskTransforms = None, preload = False, imgSize = 256):


    self.df = df
    self.transforms = transforms
    self.preload = preload
    self.imgSize = imgSize
    self.imgTransforms = imgTransforms
    self.maskTransforms = maskTransforms
    
    if self.preload:
        self.preloadData()
    
    
  def preloadData(self):
    imgList = []
    maskList = []
    
    for i in tqdm(range(len(self.df))):
            
            img = PIL.Image.open(self.df.iloc[i]['imgPath'])

            img = torchvision.transforms.Resize( (self.imgSize, self.imgSize))(img)
            imgList.append(np.array(img))

            
            mask = PIL.Image.open(self.df.iloc[i]['maskPath'])

            mask = torchvision.transforms.Resize( (self.imgSize, self.imgSize))(mask)
            maskList.append(np.array(mask))

            
    self.imgData = np.array(imgList)
    self.maskData = np.array(maskList)
    del imgList, maskList


  def __len__(self):

    return len(self.df)

  def __getitem__(self, index):
        
    if self.preload:
        img = self.imgData[index,:,:,:]
        img = PIL.Image.fromarray(img)
        
        mask = self.maskData[index,:,:,:]
        mask = PIL.Image.fromarray(mask)
        
        
    else:
        imgFilename = self.df.iloc[index]['imgPath']
        maskFilename = self.df.iloc[index]['maskPath'] 
        
        img = LoadNifti(image_only=True)(imgFilename)
        mask = LoadNifti(image_only=True)(maskFilename)
        
    ## Tumor extraction for mask
    mask = mask ==2
    mask = mask.astype('float')
    
    
    if self.imgTransforms:
        img = self.imgTransforms(img)
    
    if self.maskTransforms:
        mask = self.maskTransforms(mask)
        

#     if len(mask.shape)==4:
#         mask = mask[0,:,:,:]
    
#     if len(img.shape)==4:
#         img = img[0,:,:,:]

    mask[mask>0] = 1

    return img, mask


def evalModel( model, testLoader, threshold = 0.5):
        
    """
    Evaluation of the model is done using Dice metric
    """

    model.eval()

    diceScores = []

    for (img, mask) in tqdm(testLoader):

        img, mask = img.to(device), mask.to(device)

        out = model(img)
        out = out>threshold
        
        
        out = out.detach().cpu().numpy()
        mask = mask.detach().cpu().numpy()
        
        diceScores.append(dice(out, mask))
        
        

    model.train()

    return np.mean(diceScores)

### Training Details

In [14]:
batchSize = 3

device = 'cuda:0'

lr = 0.0001

imgSize = 64

In [15]:
imgTransforms = Compose([AddChannel(), Resize((imgSize,imgSize,imgSize)), ToTensor()])
maskTransforms = Compose([AddChannel(), Resize((imgSize,imgSize,imgSize)), ToTensor()])

trainDataset = SegDataset3D(trainDf, imgTransforms, maskTransforms, preload=False)
testDataset = SegDataset3D(testDf, imgTransforms, maskTransforms, preload=False)

In [16]:
trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=batchSize, 
                                          shuffle=True, num_workers=6)


testLoader = torch.utils.data.DataLoader(testDataset, batch_size=1, 
                                          shuffle=False, num_workers=6)

### Model

In [17]:
model = UNet3D(in_channels=1, out_channels=1, final_sigmoid = True)
model = model.to(device)

In [18]:
criterion = torch.nn.BCELoss()

# criterion = torch.nn.CrossEntropyLoss()

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=lr),
])



In [19]:
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
#         inputs = F.sigmoid(inputs)       
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

In [20]:
criterion = DiceLoss()

In [ ]:
num_epochs = 6

best_metric = 0.0
best_epoch = 0
best_model = None

for epoch in range(num_epochs):
#     self.epoch +=1

    epoch_start_time = time.time()

    model.train()


    running_loss = 0.0
    running_corrects = 0

    model.training = False
    
    for data,mask in trainLoader:
        data,mask = data.to(device), mask.to(device)

        optimizer.zero_grad()

        
        outputs = model(data)
        loss = criterion(outputs, mask.float())

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
    epoch_loss = running_loss/len(trainLoader)
            
    epochDice = evalModel(model, testLoader)
    
    print(f"Epoch : {epoch} Loss : {epoch_loss} Val Dice : {epochDice}")

100%|██████████| 27/27 [02:23<00:00,  5.31s/it]

Epoch : 0 Loss : 0.9670510700770787 Val Dice : 0.08959002261188818



100%|██████████| 27/27 [01:38<00:00,  3.64s/it]

Epoch : 1 Loss : 0.9372040663446699 Val Dice : 0.08384853445868587


In [36]:
criterion(outputs, mask.float())

tensor(0.8595, device='cuda:0', grad_fn=<RsubBackward1>)